# Field players dataframes

In [1]:
import pandas as pd
import numpy as np

In [2]:
miscel_df = pd.read_csv('csv/fplayer_scraps/miscel.csv')
miscel_df.drop(['Unnamed: 0'], axis=1, inplace=True)

others_df = pd.read_csv('csv/fplayer_scraps/others.csv')
others_df.drop(['Unnamed: 0'], axis=1, inplace=True)

shooting_df = pd.read_csv('csv/fplayer_scraps/shooting.csv')
shooting_df.drop(['Unnamed: 0'], axis=1, inplace=True)

defending_df = pd.read_csv('csv/fplayer_scraps/defending.csv')
defending_df.drop(['Unnamed: 0'], axis=1, inplace=True)

dribbling_df = pd.read_csv('csv/fplayer_scraps/dribbling.csv')
dribbling_df.drop(['Unnamed: 0'], axis=1, inplace=True)

passing_df = pd.read_csv('csv/fplayer_scraps/passing.csv')
passing_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [3]:
to_defending = miscel_df[['sofifa_id', 'cards_yellow', 'cards_red', 'cards_yellow_red', 'fouls', 'fouled', 'pens_conceded', 'own_goals', 'ball_recoveries', 'aerials_won_pct']]
to_passing = miscel_df[['sofifa_id', 'crosses']]
to_shooting = miscel_df[['sofifa_id', 'offsides', 'pens_won']]

In [4]:
defending_df = pd.merge(defending_df, to_defending, on='sofifa_id')
passing_df = pd.merge(passing_df, to_passing, on='sofifa_id')
shooting_df = pd.merge(shooting_df, to_shooting, on='sofifa_id')

In [5]:
def just_mean(right_df, left_df):
    general_df = pd.merge(right_df, left_df, on='sofifa_id')
    general_df['mean'] = round(general_df.iloc[:,1:].mean(axis=1)*10)
    general_rat = general_df.iloc[:,[0,-1]]
    return general_rat

# Shooting

In [6]:
shooting_df.head()

,sofifa_id,shots_on_target_pct,goals,shots_on_target,shots_total,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,average_shot_distance,shots_free_kicks,pens_made,pens_att,offsides,pens_won
0,20801.0,34.2,28.0,76.0,222.0,5.67,1.94,0.10,0.29,17.7,23.0,6.0,7.0,31.0,0.0
1,158023.0,48.2,51.0,123.0,255.0,5.70,2.75,0.18,0.37,18.7,52.0,5.0,6.0,19.0,1.0
2,190871.0,50.5,23.0,48.0,95.0,3.79,1.91,0.18,0.35,18.3,18.0,5.0,5.0,10.0,2.0
3,192985.0,39.2,6.0,20.0,51.0,2.52,0.99,0.12,0.30,22.9,6.0,0.0,0.0,1.0,0.0
4,155862.0,30.2,10.0,13.0,43.0,1.11,0.34,0.05,0.15,12.0,3.0,8.0,8.0,5.0,0.0


In [7]:
shooting_df['pen_acc'] = (shooting_df['pens_made'] / shooting_df['pens_att'])
shooting_df.drop(['pens_made', 'pens_att'], axis=1, inplace=True)
shooting_df.fillna(0, inplace=True)

In [8]:
rated_pos = pd.concat([shooting_df.iloc[:,0], shooting_df.iloc[:,np.r_[1:11,12,13]].apply(lambda x: pd.cut(x, bins=10, labels=list(range(1,11))))], axis=1)
rated_neg = pd.concat([shooting_df.iloc[:,0], pd.cut(shooting_df.iloc[:,11], bins=10, labels=list(range(10,0,-1)))], axis=1)

shooting_rat = just_mean(rated_pos, rated_neg)
shooting_rat.head()

,sofifa_id,mean
0,20801.0,61.0
1,158023.0,80.0
2,190871.0,61.0
3,192985.0,40.0
4,155862.0,35.0


# Defending

In [9]:
defending_df.head()

,sofifa_id,tackles,tackles_won,dribble_tackles_pct,pressure_regain_pct,blocks,interceptions,clearances,errors,cards_yellow,cards_red,cards_yellow_red,fouls,fouled,pens_conceded,own_goals,ball_recoveries,aerials_won_pct
0,20801.0,13.0,10.0,21.1,23.9,12.0,13.0,33.0,0.0,4.0,1.0,0.0,27.0,70.0,0.0,0.0,144.0,43.5
1,158023.0,23.0,20.0,20.0,26.2,20.0,8.0,0.0,0.0,3.0,0.0,0.0,31.0,101.0,0.0,0.0,189.0,36.4
2,190871.0,22.0,17.0,16.7,21.8,16.0,8.0,3.0,0.0,7.0,0.0,0.0,25.0,110.0,0.0,0.0,114.0,63.6
3,192985.0,23.0,30.0,19.2,29.4,17.0,15.0,4.0,0.0,2.0,0.0,0.0,19.0,24.0,0.0,0.0,114.0,42.1
4,155862.0,64.0,49.0,54.7,36.4,51.0,48.0,109.0,3.0,13.0,1.0,1.0,47.0,43.0,0.0,0.0,352.0,70.2


In [10]:
rated_pos = pd.concat([defending_df.iloc[:,0], defending_df.iloc[:,np.r_[1:8, 13,16,17]].apply(lambda x: pd.cut(x, bins=10, labels=list(range(1,11))))], axis=1)
rated_neg = pd.concat([defending_df.iloc[:,0], defending_df.iloc[:,np.r_[8:12, 14,15,]].apply(lambda x: pd.cut(x, bins=10, labels=list(range(10,0,-1))))], axis=1)

defending_rat = just_mean(rated_pos, rated_neg)
defending_rat.head()

,sofifa_id,mean
0,20801.0,54.0
1,158023.0,59.0
2,190871.0,57.0
3,192985.0,57.0
4,155862.0,61.0


# Dribbling

In [11]:
dribbling_df.head()

,sofifa_id,dribbles_completed_pct,players_dribbled_past,carries,carry_distance,carry_progressive_distance,miscontrols,dispossessed
0,20801.0,56.3,74.0,1646.0,12031.0,7507.0,83.0,57.0
1,158023.0,63.5,205.0,2801.0,17955.0,10930.0,67.0,107.0
2,190871.0,56.4,132.0,1570.0,11606.0,7214.0,84.0,88.0
3,192985.0,67.7,25.0,851.0,4994.0,2973.0,21.0,26.0
4,155862.0,55.6,6.0,1807.0,9332.0,5032.0,4.0,3.0


In [12]:
rated_pos = pd.concat([dribbling_df.iloc[:,0], dribbling_df.iloc[:,1:6].apply(lambda x: pd.cut(x, bins=10, labels=list(range(1,11))), axis=0)], axis=1)
rated_neg = pd.concat([dribbling_df.iloc[:,0], dribbling_df.iloc[:,6:].apply(lambda x: pd.cut(x, bins=10, labels=list(range(10,0,-1))), axis=0)], axis=1)

dribbling_rat = just_mean(rated_pos, rated_neg)
dribbling_rat.head()

,sofifa_id,mean
0,20801.0,57.0
1,158023.0,77.0
2,190871.0,59.0
3,192985.0,51.0
4,155862.0,60.0


# Passing

In [13]:
passing_df.head()

,sofifa_id,passes_pct,passes_completed,passes_total_distance,passes_progressive_distance,passes_pct_short,passes_pct_medium,passes_pct_long,assists,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,progressive_passes,crosses
0,20801.0,81.7,1166.0,18214.0,3255.0,87.4,83.7,66.9,10.0,61.0,65.0,46.0,15.0,81.0,76.0
1,158023.0,79.8,2027.0,35717.0,11972.0,89.1,78.7,65.3,19.0,114.0,243.0,183.0,11.0,384.0,89.0
2,190871.0,78.3,1099.0,18731.0,7419.0,84.6,84.2,56.7,11.0,63.0,186.0,85.0,1.0,228.0,46.0
3,192985.0,76.1,743.0,14636.0,3943.0,87.1,81.4,56.3,10.0,42.0,77.0,51.0,10.0,95.0,112.0
4,155862.0,91.4,2064.0,44851.0,17138.0,94.7,93.9,83.2,1.0,8.0,165.0,7.0,0.0,111.0,5.0


In [14]:
rated = pd.concat([passing_df.iloc[:,0], passing_df.iloc[:,1:].apply(lambda x: pd.cut(x, bins=10, labels=list(range(1,11))), axis=0)], axis=1)
rated['mean_passing'] = round(rated.iloc[:,1:].mean(axis=1)*10)

passing_rat = rated.iloc[:,[0,-1]]
passing_rat.head()

,sofifa_id,mean_passing
0,20801.0,53.0
1,158023.0,77.0
2,190871.0,56.0
3,192985.0,50.0
4,155862.0,53.0


# Impact on team

In [15]:
others_df.head()

,sofifa_id,points_per_match,plus_minus,plus_minus_wowy
0,20801.0,2.26,37.0,0.15
1,158023.0,2.32,74.0,1.13
2,190871.0,2.29,49.0,0.26
3,192985.0,2.56,50.0,0.54
4,155862.0,1.97,28.0,0.48


In [16]:
rated = pd.concat([others_df.iloc[:,0], others_df.iloc[:,1:].apply(lambda x: pd.cut(x, bins=10, labels=list(range(1,11))), axis=0)], axis=1)
rated['mean_impact'] = round(rated.iloc[:,1:].mean(axis=1)*10)

impact_rat = rated.iloc[:,[0,-1]]
impact_rat.head()

,sofifa_id,mean_impact
0,20801.0,63.0
1,158023.0,87.0
2,190871.0,70.0
3,192985.0,77.0
4,155862.0,60.0
